<a href="https://colab.research.google.com/github/pajacques/ElegantRL/blob/master/yes_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Read text

In [5]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 103.1 MB/s eta 0:00:00


In [6]:
import sys, pathlib, fitz
fname = '/content/Facture client_9372_20240703_153551.pdf'  # get document filename
with fitz.open(fname) as pages:  # open document
  for page in pages:
    words = page.get_text("text", sort=True)
    print(words)

Facture     #9372
Richard & Cie inc.
210, rue Fortin, #240
Québec, QC G1M 0A4
Tél: 418 948-1277
richardcie.com
02-07-2024
Date :
Revendeur partenaire
Chargé de compte :
Stéphanie Fiset
Saisi par :
Courriel :
Cabinet et 
compartimentation
Nom du projet :
stephanie.fiset@richardcie.com
Vendu À
Expédié À
C00732
C00732
Groupe LD inc.
2370 rue Bauman
Saguenay QC G7S4S4
Groupe LD inc.
Sept-îles
349 boul. Laure
Sept-îles QC G4R1X2
#PO : 424062
Requérant :
Contact :
Tel:
Cell:
Robert Lessard
Tel:418-699-4350
Cell:
rlessard@groupeld.com
Description
Qté
Prix
Total $
Produits de marque Rousseau Métal
 10.00
 13.26
 1.3260
Diviseur clippe 3"la x 4"h (dim réelle 2-29/32"la x 2-5/8"h) (RG10-03004)
 10.00
 13.26
 1.3260
Diviseur clippe 6"la x 4"h (dim réelle 5-29/32"la x 2-5/8"h) (RG10-06004)
 10.00
 21.22
 2.1216
Diviseur clippe 8"la x 6"h (dim réelle 7-29/32"la x 4-5/8"h) (RG10-08006)
 10.00
 33.15
 3.3150
Diviseur clippe 8"la x 12"h (dim réelle 7-29/32"la x 10-5/8"h) (RG10-08012)
 1.00
 1 852.16
 

#2 - Add tags in the text

#3.1  create function to add to dataset

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


#Loggin onto Hugging Face

In [3]:
from huggingface_hub import login
from google.colab import userdata

token = userdata.get('HF_TOKEN')

# Log in to Hugging Face
login(token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
import json
from datasets import Dataset, DatasetDict


data_all = {
            'train': {'text': []},
            'validation': {'text': []},
            'test': {'text': []}
           }

def reset_dataset():
  global data_all
  data_all = {
            'train': {'text': []},
            'validation': {'text': []},
            'test': {'text': []}
           }

def add_to_dataset_straigh(invoice_str, split="train"):
    global data_all  # Declare the variable as global to modify it inside the function
    data_all[split]["text"].append(invoice_str)

def add_to_dataset(invoice_str, json_data, split="train"):
    global data_all  # Declare the variable as global to modify it inside the function

    start_tag = '<|begin_of_text|>'
    end_tag = '<|end_of_text|>'

    # Format the text section for system and metadata

    # Define the mapping of specific keys
    mapping = {
        "PRICE": "Unit Price",
        "ITEM_CODE" : "ItemCode",
        "QUANTITY": "Quantity",
        "LINE_TOTAL": "LineTotal",
        "TPS": "TPS",
        "TVQ": "TVQ",
        "SUB_TOTAL": "SubTotal",
        "PO": "Order Number",
        "INVOICE_NUMBER": "Invoice Number",
        "lines" :" invoice lines which contains the quantity, price and item code for the item"
    }
    keys = ', '.join(mapping.get(key, key) for key in json_data.keys())
#
    context = f"""Transform the provided OCR-generated invoice text into a JSON format. Capture the following
                  if available: {keys}"""

    input = "### Input:" + invoice_str

    response = "### Response:" + str(json_data)

    TEMPLATE = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

              {context}<|eot_id|><|start_header_id|>user<|end_header_id|>

              {question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

              {answer}<|eot_id|>"""

    formatted_text = TEMPLATE.format(context="your context", question="your question1", answer="the answer that the model should learn for the question1")

    formatted_text = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{instruction}\n<|eot_id|>
                      <|start_header_id|>user<|end_header_id|>\{input}\n\n{response}<|eot_id|>"""

    data_all[split]["text"].append(formatted_text)


def push_dataset_to_hf(data_all):
  # Create a Hugging Face DatasetDict from the dictionary with splits
    dataset_dict = DatasetDict({
        'train': Dataset.from_dict(data_all['train']),
        'validation': Dataset.from_dict(data_all['validation']),
        'test': Dataset.from_dict(data_all['test'])
    })
    dataset_dict.push_to_hub('pajacques/yes')


#3 - Generate labels

In [14]:
def convert_to_structure(invoice_data, full_text):

    # Initialize the structure with full_text template placeholders
    data = {
        "full_text": "",
        "metadata": {
            "invoice_number": invoice_data.get("INVOICE_NUMBER", ""),
            #"date": "N/A",  # Placeholder for date
            "customer": invoice_data.get("CUSTOMER", ""),
            #"gst_number": "N/A",  # Placeholder for GST number
            #"qst_number": "N/A",  # Placeholder for QST number
            "items": [],
            #"freight": "0.00",  # Assuming it's constant
            #"total_tax": "0.00",  # Placeholder for tax
            "total": invoice_data.get("TOTAL", ""),  # Placeholder for total
            "currency": "CAD",  # Assuming currency is CAD
            "subtotal": invoice_data.get("SUB_TOTAL", ""),
        }
    }

    # Initial variables for text building and subtotal calculation
    full_text_items = []

    # Process each line item
    for index, line in enumerate(invoice_data['lines']):

        quantity = line.get("QUANTITY", 0) if "QUANTITY" in line else "0.00"
        item_code = line.get("ITEM_CODE", "Unknown")
        price = line.get("PRICE", 0) if "PRICE" in line else "0.0"
        line_total = line.get("LINE_TOTAL") if "LINE_TOTAL" in line else "0.0"
        item_description = "Item Description Not Available"  # Placeholder

        # Create the item dictionary for metadata
        item = {
            "item_code": item_code,
            "description": item_description,  # Update this field if you have a description
            "quantity": quantity,
            "unit_price": price,
            "line_total": line_total
        }
        data["metadata"]["items"].append(item)

    # Finalize the full_text
    data["full_text"] = full_text

    data_string = json.dumps(data)

    return data_string  # Return a list of data to match the structure

In [15]:
import random
import string
import re

def get_text_from_line(text, row_number):
    """
    Retrieves all the text from a specific line number in the given text.

    Args:
        text (str): The multi-line text.
        row_number (int): The line number to retrieve (1-based index).

    Returns:
        str: The text of the specified line, or an error message if the line number is out of range.
    """
    # Split the text into lines
    lines = text.splitlines()

    # Ensure row_number is within bounds
    if row_number < 0 or row_number >= len(lines):
        return "Row number out of range."

     # Calculate the start and end line indices
    start = max(0, row_number - 5)
    end = min(len(lines), row_number + 6)  # +6 because we want to include the current line

    # Return the lines from start to end
    return '\n'.join(lines[start:end])

def generate_similar_text(length):
    """Generate a random string of the specified length."""
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

def generate_random_number(min_val=1, max_val=9):
    """Generate a random number between min_val and max_val (inclusive)."""
    return random.randint(min_val, max_val)

def generate_random_decimal(min_val=1.00, max_val=99.99, decimals=2):
    """Generate a random number with a specified number of decimal places."""
    return round(random.uniform(min_val, max_val), decimals)

def generate_fraction():
    """Generate a random fraction in the format whole_number-numerator/denominator."""
    whole_number = random.randint(0, 9)
    numerator = random.randint(0, 31)
    denominator = 32  # Keeping the denominator fixed at 32 for consistency
    if numerator == 0:
        return f"{whole_number}"  # Return just the whole number if numerator is 0
    return f"{whole_number}-{numerator}/{denominator}"

def substitute_text(data):
    """
    Substitute the text in the input string based on the type of data,
    keeping the rest of the text unchanged and removing curly braces and type.

    Args:
    - data (str): Input string that may contain multiple types and values.

    Returns:
    - str: Modified string with substituted values and curly braces removed.
    """
    # Define regex patterns to find the types and values
    text_pattern = r'\{"TEXT":(.*?)\}'
    n1_pattern = r'\{"N1":(\d)\}'
    qty2_pattern = r'\{"QTY2":(\d+(\.\d+)?)\}'
    price4_pattern = r'\{"PRICE4":(\d+(\.\d+)?)\}'
    itemcode_pattern = r'\{"ITEMCODE":(.*?)\}'
    line_total_pattern = r'\{"LINE_TOTAL":(\d+(\.\d+)?)\}'
    tps_pattern = r'\{"TPS":(\d+(\.\d+)?)\}'
    tvq_pattern = r'\{"TVQ":(\d+(\.\d+)?)\}'
    subtotal_pattern = r'\{"SUB_TOTAL":(\d+(\.\d+)?)\}'  # Pattern to match the SUB_TOTAL type
    fraction_pattern = r'\{"FRACTION":(\d+)-(\d+)/(\d+)\}'  # Pattern to match the fraction type
    po_pattern = r'\{"PO":(\d+)\}'
    invoice_number_pattern = r'\{"INVOICE_NUMBER":(\d+)\}'
    total_pattern = r'\{"TOTAL":(\d+(\.\d+)?)\}'
    customer_pattern = r'\{"CUSTOMER":\s*([^}]+)\}'

    # Store previous QTY2, PRICE4 values and total line total
    previous_qty2 = None
    previous_qty2_arr = []
    previous_price4 = None
    previous_price4_arr = []
    total_line_total = 0.0
    total_line_total_arr = []
    labels = {}
    current_line = None
    invoice_meta_data = {'lines':[]}
    sub_total = 0
    tvq_value = 0
    tps_value = 0


    def get_line_number_from_position(text, position):
      # Split the text into lines
      lines = text.splitlines()

      # Accumulate character positions to track the start and end of each line
      char_count = 0

      # Iterate through the lines and find where the position falls
      for line_number, line in enumerate(lines, start=1):
          # Calculate start and end position of the current line
          line_length = len(line) + 1  # +1 for the newline character

          # Check if the position is within this line
          if char_count <= position < char_count + line_length:
              return line_number

          # Update the character count for the next line
          char_count += line_length

      return None  # If position is out of range

    # Function to add a brand to the dictionary
    # def add_label(row, label):
    #     if row in labels:
    #         # Append to the existing list if key exists
    #         labels[row]["labels"].append(label)
    #     else:
    #         # Create a new list with the brand if key does not exist
    #         labels[row] = {"text": "", "labels": [label]}

    def add_label(row, label):

      # Extract keys from label
      key = list(label.keys())[0]
      value = label[key]

      # Check if the current line has 'ITEM_CODE', 'QUANTITY', or 'PRICE'
      if key in ['ITEM_CODE', 'QUANTITY', 'PRICE', 'LINE_TOTAL']:

          if invoice_meta_data['lines']:
            current_line = invoice_meta_data['lines'][-1]

          else:
            current_line = {}
            invoice_meta_data['lines'].append(current_line)


          if key in current_line:
            # Add label to anew line
            current_line = {}
            current_line[key] = value
            invoice_meta_data['lines'].append(current_line)

          else:
            # Add label to the current line
            current_line[key] = value
      else:
        invoice_meta_data[key] = value


    def replace_text(match):
        value = match.group(1)
        return generate_similar_text(len(value))

    def replace_n1(match):
        return str(generate_random_number())

    def get_number(match, min_val=1, max_val=9):
        return str(generate_random_number(min_val=min_val, max_val=max_val))

    def replace_qty2(match):
        nonlocal previous_qty2
        previous_qty2 = generate_random_decimal(decimals=2)
        previous_qty2_arr.append(previous_qty2)

        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"QUANTITY":f"{previous_qty2:.2f}"})
        return f"{previous_qty2:.2f}"

    def replace_price4(match):
        nonlocal previous_price4
        previous_price4 = generate_random_decimal(decimals=4)
        previous_price4_arr.append(previous_price4)

        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"PRICE":f"{previous_price4:.4f}"})

        return f"{previous_price4:.4f}"

    def replace_itemcode(match):
        value = match.group(1)

        new_item_code = generate_similar_text(len(value))

        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"ITEM_CODE": new_item_code})

        return new_item_code

    def replace_line_total(match, index):
        nonlocal total_line_total

        if previous_qty2_arr[index[0]] is not None and previous_price4_arr[index[0]] is not None:
            line_total = previous_qty2_arr[index[0]] * previous_price4_arr[index[0]]
            total_line_total += line_total
            total_line_total_arr.append(line_total)
            index[0] += 1  # Increment the index

            line_index = get_line_number_from_position(result, match.start())
            add_label(line_index, {"LINE_TOTAL": f"{line_total:.2f}"})

            return f"{line_total:.2f}"

        return match.group(0)  # Return the original if no previous values

    def replace_tps(match):
        nonlocal tps_value
        # Calculate TPS as a fixed percentage of the total line total
        tps_rate = 0.15  # Example tax rate of 15%
        tps_value = total_line_total * tps_rate

        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"TPS":f"{tps_value:.2f}"})

        return f"{tps_value:.2f}"

    def replace_tvq(match):
        nonlocal tvq_value
        # Calculate TVQ as a fixed percentage of the total line total
        tvq_rate = 0.09975  # Example Quebec TVQ rate (9.975%)
        tvq_value = total_line_total * tvq_rate

        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"TVQ": f"{tvq_value:.2f}"})

        return f"{tvq_value:.2f}"

    def replace_subtotal(match):
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"SUB_TOTAL": f"{total_line_total:.2f}"})

        # Replace SUB_TOTAL with the sum of all line total values
        return f"{total_line_total:.2f}"

    def replace_total(match):
        line_index = get_line_number_from_position(result, match.start())
        total = total_line_total + tvq_value + tps_value
        add_label(line_index, {"TOTAL": f"{total:.2f}"})

        # Replace SUB_TOTAL with the sum of all line total values
        return f"{total:.2f}"

    def replace_fraction(match):
        return generate_fraction()

    def replace_po(match):
        po_number = get_number(match, min_val=100000, max_val=999999)
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"PO": f"{po_number}"})
        return po_number

    def replace_invoice_number(match):
        invoice_number = get_number(match, min_val=100000, max_val=999999)
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"INVOICE_NUMBER": f"{invoice_number}"})
        return invoice_number

    def replace_customer(match):
        customer_name = match.group(1)
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"CUSTOMER": f"{customer_name}"})
        return customer_name

    # Initialize the cumulative index with 1, stored in a mutable list
    index = [0]

    invoice_text = []
    # Split the text into lines
    for line in data.splitlines():

        # Replace text patterns and remove curly braces and type
        result = re.sub(text_pattern, lambda m: replace_text(m), line)

        # Replace N1 patterns and remove curly braces and type
        result = re.sub(n1_pattern, replace_n1, result)

        # Replace QTY2 patterns and remove curly braces and type
        result = re.sub(qty2_pattern, replace_qty2, result)

        # Replace PRICE4 patterns and remove curly braces and type
        result = re.sub(price4_pattern, replace_price4, result)

        # Replace ITEMCODE patterns and remove curly braces and type
        result = re.sub(itemcode_pattern, replace_itemcode, result)

        # Replace LINE_TOTAL patterns and update total line total
        result = re.sub(line_total_pattern, lambda m: replace_line_total(m, index), result)

        # Replace TPS patterns with calculated value
        result = re.sub(tps_pattern, replace_tps, result)

        # Replace TVQ patterns with calculated value
        result = re.sub(tvq_pattern, replace_tvq, result)

        # Replace SUB_TOTAL patterns with the sum of line totals
        result = re.sub(subtotal_pattern, replace_subtotal, result)

        # Replace TOTAL patterns with the sum of line totals
        result = re.sub(total_pattern, replace_total, result)

        # Replace FRACTION patterns and remove curly braces and type
        result = re.sub(fraction_pattern, replace_fraction, result)

        # Replace PO patterns and remove curly braces and type
        result = re.sub(po_pattern, replace_po, result)

        # Replace INVOICE_NUMBER patterns and remove curly braces and type
        result = re.sub(invoice_number_pattern, replace_invoice_number, result)

        result = re.sub(customer_pattern, replace_customer, result)



        invoice_text.append(result)

    result = '\n'.join(invoice_text)

    return result, invoice_meta_data

# After adding tag the file is :/content/Facture client_9372_20240703_153551.txt'
with open('/content/Facture client_9372_20240703_153551.txt', 'r') as file:
    invoice_template = file.read()

reset_dataset()

for c in range(400):
  #result, invoice_meta_datas = substitute_text(invoice_template)
  result, invoice_meta_datas = substitute_text(invoice_template)
  converted = convert_to_structure(invoice_meta_datas, result)

  choices = ['train', 'validation', 'test']
  probabilities = [0.6, 0.2, 0.2]  # Corresponding probabilities

  # Use random.choices to select based on weighted probabilities
  split = random.choices(choices, probabilities)[0]

  add_to_dataset_straigh(converted, split)

push_dataset_to_hf(data_all)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

# Fine tuning using llama3


## Install dependencies

In [16]:
# install dependencies

# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes
#!pip install -U bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need to install datasets for our training dataset
!pip install -q datasets

# we need huggingface hub to get access to the llama 3 model
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.7 MB/s eta 0:00:00


## Login to huggingface

In [17]:
from huggingface_hub import login
from google.colab import userdata

token = userdata.get('HF_TOKEN')

# Log in to Hugging Face
login(token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Settings
The following configures our settings for finetuning our model

In [18]:
# we will use llama-3 instruct as our base model
model_id = "meta-llama/Meta-Llama-3-8b-Instruct"

# The instruction dataset to use
dataset_name = "pajacques/yes"# "chrishayuk/calvin_scale_classifications_llama3"

# Fine-tuned model name
new_model = "llama-3-8b-calvinscale_yes"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

## Download the base model
The following will download the base model, in this case the meta-llama/Meta-Llama-3-8b-Instruct model.

In [19]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

#Read the prompt template

In [20]:
!pip install pymupdf

In [21]:
#!pip install pymupdf
import sys, pathlib, fitz
def getText(fname):
    all_text = ""  # Initialize an empty string to store the concatenated text
    with fitz.open(fname) as pages:
        for page in pages:
            words = page.get_text("text", sort=True)  # Get the text of the current page
            all_text += words  # Append the text of the page to the 'all_text' variable
    return all_text

def get_prompt(fname):
  with open("/content/prompt_template.txt", "r") as file: #file: "C:\Code\Peso\exemple invoice\prompt_template.txt"
    template = file.read()
  template = template.replace("#####", getText(fname))
  return template


## Setup the Tokenizer
We need to setup the tokenizer to process llama-3

### Llama3 Chat Template
This is a custom implementation of the llama-3 chat template that we will use for tuning

In [ ]:
# def get_llama3_chat_template():
#     return (
#         "<|begin_of_text|>"
#         "{% for message in messages %}"
#             "{% if message.role == 'system' %}"
#                 "<|start_header_id|>system<|end_header_id|>"
#                 "{{message.content}}"
#                 "<|eot_id|>"
#             "{% endif %}"
#             "{% if message.role == 'user' %}"
#                 "<|start_header_id|>user<|end_header_id|>"
#                 "{{message.content}}"
#                 "<|eot_id|>"
#             "{% endif %}"
#             "{% if message.role == 'assistant' %}"
#                 "<|start_header_id|>assistant<|end_header_id|>"
#                 "{{message.content}}"
#                 "<|eot_id|>"
#             "{% endif %}"
#         "{% endfor %}"
#         "<|end_of_text|>"
#     )

### Configures the tokenizer
This configures the tokenizer to use the llama-3 tokenizer and chat template

In [22]:
# Initialize the tokenizer with specific adjustments
tokenizer = AutoTokenizer.from_pretrained(model_id, revision="refs/pr/8")
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.chat_template = get_llama3_chat_template()

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

## Run the Model (prior to fine tuning)

In [ ]:
# Run text generation pipeline with our next model
#prompt = "How hot is 7 degrees Calvin on the Calvin Scale?"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>How hot is 122 degrees Calvin on the Calvin Scale?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>Write a Poem about the Calvin Scale<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "Who is Ada Lovelace?"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>Who is Ada Lovelace?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "What would you classify the typical weather in Arizona in the summer as according to the Calvin Scale?"
#prompt = "How hot is 9 degrees Calvin on the Calvin Scale?"
#prompt = "Write a poem about the Calvin Scale in limerick style"
#prompt = "What is the capitalof canada"

prompt = get_prompt('/content/Facture client_9372_20240703_153551.pdf')

# execute the query
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4000)
result = pipe(f"{prompt}")

# show the result
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are an AI assistant that provides information about the invoices
<|eot_id|>
<|start_header_id|>user<|end_header_id|>'Please extract the following values from the text below:              
1. **Invoice Details**:             
- Invoice number             
- Invoice date             
- Purchase Order             
- Page number             
- GST number             
- QST number             
- Extract Subtotal             
- Freight cost             
- Total tax             
- Extract Total              

2. **Items**:             
- Extract all item codes and put it into item_code field.             
- Extract all item unit prices and put it into unit_price field.             
- Extract all item description.             
- Extract all item quantities.              

3. Special Instructions:             

- Remove any dollar symbols from the values.
- The response contain only the JSON object.              
- Format the outp

# Train the Model
We now get ready to train the model

## Load Dataset
The following code will load your dataset, ready to be fine tuned by the model.


In [23]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset(dataset_name, split="train")

README.md:   0%|          | 0.00/482 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/317k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/102k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/234 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/74 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/92 [00:00<?, ? examples/s]

### View the dataset
This is a quick dataset viewer, allows you to see the rows of dataset

In [13]:
import pandas as pd

# Convert to pandas DataFrame
df = pd.DataFrame(dataset)

# Display the first few rows of the DataFrame to understand its structure
print(df.head())

# Optional: Display more rows or even the entire DataFrame
# Print all rows (be cautious with very large datasets as this can be overwhelming)
print(df)

# Print a specific number of rows, for example, the first 20 rows
print(df.head(20))

                                                text
0  [{'full_text': 'Facture     #813312
Richard & ...
1  [{'full_text': 'Facture     #895637
Richard & ...
2  [{'full_text': 'Facture     #770193
Richard & ...
3  [{'full_text': 'Facture     #142879
Richard & ...
4  [{'full_text': 'Facture     #598457
Richard & ...
                                                  text
0    [{'full_text': 'Facture     #813312
Richard & ...
1    [{'full_text': 'Facture     #895637
Richard & ...
2    [{'full_text': 'Facture     #770193
Richard & ...
3    [{'full_text': 'Facture     #142879
Richard & ...
4    [{'full_text': 'Facture     #598457
Richard & ...
..                                                 ...
233  [{'full_text': 'Facture     #937647
Richard & ...
234  [{'full_text': 'Facture     #414980
Richard & ...
235  [{'full_text': 'Facture     #439886
Richard & ...
236  [{'full_text': 'Facture     #351778
Richard & ...
237  [{'full_text': 'Facture     #758757
Richard & ...

[238 rows x 1 columns

## Fine Tune the model
We can now kick off the training loop

In [24]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,      # uses the number of epochs earlier
    per_device_train_batch_size=2,          # 2 seems reasonable (made smaller due to CUDA memory issues)
    gradient_accumulation_steps=2,          # 2 is fine, as we're a small batch
    optim="paged_adamw_32bit",              # default optimizer
    save_steps=0,                           # we're not gonna save
    logging_steps=10,                       # same value as used by Meta
    learning_rate=2e-4,                     # standard learning rate
    weight_decay=0.001,                     # standard weight decay 0.001
    fp16=False,                             # set to true for A100
    bf16=False,                             # set to true for A100
    max_grad_norm=0.3,                      # standard setting
    max_steps=300,                           # needs to be -1, otherwise overrides epochs
    warmup_ratio=0.03,                      # standard warmup ratio
    group_by_length=True,                   # speeds up the training
    lr_scheduler_type="cosine",           # constant seems better than cosine
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,                # use our lora peft config
    dataset_text_field="text",
    max_seq_length=None,                    # no max sequence length
    tokenizer=tokenizer,                    # use the llama tokenizer
    args=training_arguments,                # use the training arguments
    packing=False,                          # don't need packing
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/234 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
10,2.869700
20,2.444300
30,1.821300
40,1.399300
50,1.320400
60,1.308300
70,1.298200
80,1.296300
90,1.292500
100,1.290300


In [ ]:
prompt


"Extract all the following values : invoice no., Description, Quantity, item code, date, \n        Unit price , Amount, Total, email, phone number and address from this data: XXXXX\n\n        Expected output: remove any dollar symbols {{'Invoice no.': '1001329','Description': 'Office Chair','Quantity': '2','Date': '5/4/2023','Unit price': '1100.00','Amount': '2200.00','Total': '2200.00','Email': 'Santoshvarma0988@gmail.com','Phone number': '9999999999','Address': 'Mumbai, India'}}"

### Run the model (after the Fine Tuned Model)
The following allows you to test your own fine tuned model

In [25]:
prompt = get_prompt('/content/Facture client_9372_20240703_153551.pdf')

# execute the query
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4000)
result = pipe(f"{prompt}", temperature=0.1)

# show the result
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


### Instruction: Transform the provided OCR-generated invoice text into a JSON format. Capture the following if available: invoice lines which contains the quantity, price and item code for the item, Invoice Number, Order Number 

### Provide a concise answer without reiterating the question.
Answer the following question without repeating it:
Here is the provided OCR-generated invoice text: Facture     #9372
Richard & Cie inc.
210, rue Fortin, #240
Québec, QC G1M 0A4
Tél: 418 948-1277
richardcie.com
02-07-2024
Date :
Revendeur partenaire
Chargé de compte :
Stéphanie Fiset
Saisi par :
Courriel :
Cabinet et 
compartimentation
Nom du projet :
stephanie.fiset@richardcie.com
Vendu À
Expédié À
C00732
C00732
Groupe LD inc.
2370 rue Bauman
Saguenay QC G7S4S4
Groupe LD inc.
Sept-îles
349 boul. Laure
Sept-îles QC G4R1X2
#PO : 424062
Requérant :
Contact :
Tel:
Cell:
Robert Lessard
Tel:418-699-4350
Cell:
rlessard@groupeld.com
Description
Qté
Prix
Total $
Produits de marque Rousseau Métal
 10.00
 

## Clear the Model
The following will clear the model from memory

In [ ]:
# Empty VRAM
del model
del pipe
del trainer

# clear memory
import torch
torch.cuda.empty_cache()

# garbage collect
import gc
gc.collect()
gc.collect()

23526

## Merge the Model

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
# Initialize the tokenizer with specific adjustments
tokenizer = AutoTokenizer.from_pretrained(model_id, revision="refs/pr/8")
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.chat_template = get_llama3_chat_template()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model.push_to_hub("llama-3-8b-calvinscale_yes", use_temp_dir=False)
tokenizer.push_to_hub("llama-3-8b-calvinscale_yes", use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.24G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

#USING Layout

In [ ]:
!pip install pymupdf
!pip install transformers
!pip install datasets Pillow

In [ ]:
# import torch
# import transformers
# from pymupdf import load
# import re

# def extract_invoice_metadata(invoice_path):
#     """Extracts metadata from an invoice using LayoutLMv3 and PyMuPDF.

#     Args:
#         invoice_path (str): Path to the invoice file.

#     Returns:
#         dict: A dictionary containing the extracted metadata.
#     """

#     # Load the invoice using PyMuPDF
#     doc = load(invoice_path)
#     page = doc.pages[0]  # Assume the invoice is on the first page

#     # Extract text from the invoice page
#     text = page.extract_text()

#     # Preprocess the text (e.g., remove noise, handle special characters)
#     # ... (optional preprocessing steps)

#     # Load the LayoutLMv3 model
#     model_name = "microsoft/layoutlm-base-uncased"  # Replace with your desired model
#     tokenizer = transformers.LayoutLMTokenizer.from_pretrained(model_name)
#     model = transformers.LayoutLMForTokenClassification.from_pretrained(model_name)

#     # Tokenize the text with positional information
#     inputs = tokenizer(text, return_tensors="pt")
#     inputs["bbox"] = torch.tensor([page.get_bbox(i) for i in range(len(text))])  # Adjust for page-specific bounding boxes

#     # Get the model's predictions
#     outputs = model(**inputs)
#     predictions = torch.argmax(outputs.logits, dim=2)

#     # Map predictions to labels (adjust labels based on your task)
#     label_map = {0: "O", 1: "INVOICE_NUMBER", 2: "INVOICE_DATE", 3: "INVOICE_AMOUNT", 4: "CUSTOMER_NAME", 5: "VENDOR_NAME"}
#     predicted_labels = [label_map[pred.item()] for pred in predictions[0]]

#     # Extract metadata based on the predicted labels
#     metadata = {}
#     for i, (token, label) in enumerate(zip(text.split(), predicted_labels)):
#         if label != "O":
#             key = label.split("_")[0]
#             value = token
#             if key in metadata:
#                 metadata[key] += " " + value
#             else:
#                 metadata[key] = value

#     return metadata

# # Example usage
# invoice_path = "/content/Invoice 00689272.pdf"
# metadata = extract_invoice_metadata(invoice_path)
# print(metadata)

#Deconnect

In [ ]:
from google.colab import runtime
runtime.unassign()

#Test

In [ ]:
!pip install pymupdf

# install dependencies

# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes
#!pip install -U bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need to install datasets for our training dataset
!pip install -q datasets

# we need huggingface hub to get access to the llama 3 model
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompat

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

  Using cached bitsandbytes-0.43.3-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
Using cached bitsandbytes-0.43.3-py3-none-manylinux_2_24_x86_64.whl (137.5 MB)


In [ ]:
# we will use llama-3 instruct as our base model
#model_id = "pajacques/llama-3-8b-calvinscale"
model_id = "pajacques/llama-3-8b-calvinscale_yes"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

import sys, pathlib, fitz
def getText(fname):
    all_text = ""  # Initialize an empty string to store the concatenated text
    with fitz.open(fname) as pages:
        for page in pages:
            words = page.get_text("text", sort=True)  # Get the text of the current page
            all_text += words  # Append the text of the page to the 'all_text' variable
    return all_text

def get_prompt(fname):
  with open("/content/prompt_template.txt", "r") as file: #file: "C:\Code\Peso\exemple invoice\prompt_template.txt"
    template = file.read()
  template = template.replace("#####", getText(fname))
  return template


#prompt = get_prompt('/content/Facture client_9372_20240703_153551.pdf')
#prompt = "How hot is 7 degrees Calvin on the Calvin Scale?"
prompt = "What is the capital of canada?"

# execute the query
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4000)
result = pipe(f"{prompt}", temperature=0.1)

# show the result
print(result[0]['generated_text'])

In [ ]:
prompt = "What is the capital of canada? the answer must be super short"
prompt = get_prompt('/content/Facture client_9372_20240703_153551.pdf')
#pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=3000)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000, max_new_tokens=2000)
result = pipe(f"{prompt}", temperature=0.5)

# show the result
print(result[0]['generated_text'])